# Wvlet Python SDK Tutorial

This notebook provides an interactive tutorial for the Wvlet Python SDK, demonstrating how to use Wvlet's flow-style query language to write more intuitive and maintainable SQL queries.

## Installation

First, install the Wvlet Python SDK:

In [ ]:
# Install Wvlet (uncomment if not already installed)
# !pip install wvlet

# Import the library
from wvlet import compile
from wvlet.compiler import WvletCompiler

## 1. Basic Query Compilation

Let's start with simple query examples to understand Wvlet's flow-style syntax.

In [ ]:
# Simple SELECT query
wvlet_query = "from employees select name, department, salary"
sql = compile(wvlet_query)
print("Wvlet Query:")
print(wvlet_query)
print("\nCompiled SQL:")
print(sql)

In [ ]:
# Query with WHERE clause
wvlet_query = """
from employees 
where salary > 50000 and department = 'Engineering'
select name, salary
order by salary desc
"""
sql = compile(wvlet_query)
print("Filtered query SQL:")
print(sql)

## 2. Aggregations and Grouping

Wvlet makes aggregations more readable by following a logical flow.

In [ ]:
# Department statistics
wvlet_query = """
from employees
group by department
agg 
    count(*) as employee_count,
    avg(salary) as avg_salary,
    min(salary) as min_salary,
    max(salary) as max_salary
having count(*) > 5
order by avg_salary desc
"""

sql = compile(wvlet_query)
print("Aggregation SQL:")
print(sql)

## 3. Joining Tables

Wvlet supports various join types with intuitive syntax.

In [ ]:
# Inner join example
wvlet_query = """
from orders o
join customers c on o.customer_id = c.id
join products p on o.product_id = p.id
where o.order_date >= '2024-01-01'
select 
    o.order_id,
    c.name as customer_name,
    p.name as product_name,
    o.quantity,
    p.price * o.quantity as total_amount
"""

sql = compile(wvlet_query)
print("Join query SQL:")
print(sql)

## 4. Models (CTEs)

Models in Wvlet are like CTEs (Common Table Expressions) but more intuitive to write and compose.

In [ ]:
# Using models for complex analysis
wvlet_query = """
model HighValueCustomers = {
    from orders
    group by customer_id
    agg sum(total_amount) as lifetime_value
    having sum(total_amount) > 10000
}

model RecentOrders = {
    from orders
    where order_date > current_date - 30
    group by customer_id
    agg count(*) as recent_order_count
}

from HighValueCustomers hvc
join RecentOrders ro on hvc.customer_id = ro.customer_id
select 
    hvc.customer_id,
    hvc.lifetime_value,
    ro.recent_order_count
order by lifetime_value desc
"""

sql = compile(wvlet_query)
print("Complex query with models:")
print(sql)

## 5. Window Functions

Wvlet supports all standard SQL window functions with cleaner syntax.

In [ ]:
# Ranking and running totals
wvlet_query = """
from sales
select 
    date,
    product_id,
    amount,
    row_number() over (partition by product_id order by amount desc) as rank_by_product,
    sum(amount) over (order by date) as running_total,
    avg(amount) over (
        partition by product_id 
        order by date 
        rows between 6 preceding and current row
    ) as moving_avg_7days
"""

sql = compile(wvlet_query)
print("Window functions SQL:")
print(sql)

## 6. Advanced Features

### Pivot Operations

In [ ]:
# Pivot example
wvlet_query = """
from sales
group by product_category
pivot sum(amount) for month in ('Jan', 'Feb', 'Mar', 'Apr')
"""

sql = compile(wvlet_query)
print("Pivot SQL:")
print(sql)

### UNNEST Operations

In [ ]:
# Working with arrays
wvlet_query = """
from users
cross join unnest(tags) as tag
where tag in ('premium', 'active')
select user_id, email, tag
"""

sql = compile(wvlet_query)
print("UNNEST SQL:")
print(sql)

## 7. Target-Specific Compilation

Wvlet can generate optimized SQL for different database engines.

In [ ]:
# Same query, different targets
wvlet_query = """
from logs
where timestamp > current_timestamp - interval '1 hour'
group by date_trunc('minute', timestamp) as minute
agg count(*) as event_count
"""

# Compile for different databases
for target in ['duckdb', 'trino']:
    compiler = WvletCompiler(target=target)
    sql = compiler.compile(wvlet_query)
    print(f"\n{target.upper()} SQL:")
    print(sql)

## 8. Real-World Example: Customer Analytics Dashboard

Let's build a complete customer analytics query using Wvlet.

In [ ]:
# Comprehensive customer analytics
wvlet_query = """
-- Define customer segments based on behavior
model CustomerMetrics = {
    from orders
    group by customer_id
    agg 
        count(*) as order_count,
        sum(total_amount) as lifetime_value,
        avg(total_amount) as avg_order_value,
        max(order_date) as last_order_date,
        datediff('day', max(order_date), current_date) as days_since_last_order
}

model CustomerSegments = {
    from CustomerMetrics
    select 
        customer_id,
        lifetime_value,
        order_count,
        case 
            when lifetime_value > 10000 then 'VIP'
            when lifetime_value > 5000 then 'High Value'
            when lifetime_value > 1000 then 'Regular'
            else 'Low Value'
        end as value_segment,
        case
            when days_since_last_order <= 30 then 'Active'
            when days_since_last_order <= 90 then 'At Risk'
            when days_since_last_order <= 180 then 'Dormant'
            else 'Churned'
        end as activity_segment
}

-- Analyze segments
from CustomerSegments
group by value_segment, activity_segment
agg 
    count(*) as customer_count,
    sum(lifetime_value) as total_revenue,
    avg(lifetime_value) as avg_customer_value,
    avg(order_count) as avg_orders_per_customer
order by value_segment, activity_segment
"""

sql = compile(wvlet_query)
print("Customer Analytics SQL:")
print(sql)

## 9. Integration with Pandas and DuckDB

Here's how to use Wvlet-generated SQL with actual data processing.

In [ ]:
# Example with DuckDB (if installed)
try:
    import duckdb
    import pandas as pd
    import numpy as np
    
    # Create sample data
    np.random.seed(42)
    df_sales = pd.DataFrame({
        'date': pd.date_range('2024-01-01', periods=100, freq='D'),
        'product': np.random.choice(['A', 'B', 'C'], 100),
        'amount': np.random.uniform(100, 1000, 100).round(2),
        'quantity': np.random.randint(1, 10, 100)
    })
    
    # Register DataFrame with DuckDB
    conn = duckdb.connect(':memory:')
    conn.register('sales_data', df_sales)
    
    # Use Wvlet to generate analysis query
    wvlet_query = """
    from sales_data
    group by product
    agg 
        count(*) as num_sales,
        sum(amount) as total_revenue,
        avg(amount) as avg_sale,
        sum(quantity) as total_units
    order by total_revenue desc
    """
    
    sql = compile(wvlet_query, target='duckdb')
    result = conn.execute(sql).fetchdf()
    
    print("Product Performance:")
    print(result)
    
except ImportError:
    print("DuckDB not installed. Install with: pip install duckdb")

## 10. Error Handling and Best Practices

In [ ]:
from wvlet.compiler import CompilationError

# Example of handling compilation errors
queries = [
    "from users select *",  # Valid
    "select * where id = 1",  # Missing FROM
    "from users select count(*) group",  # Incomplete GROUP BY
]

for query in queries:
    try:
        sql = compile(query)
        print(f"✓ Valid query compiled successfully")
    except CompilationError as e:
        print(f"✗ Compilation error: {e}")
    except Exception as e:
        print(f"✗ Unexpected error: {type(e).__name__}: {e}")

## Summary

This tutorial covered:

1. **Basic Syntax**: Wvlet's flow-style query structure
2. **Aggregations**: GROUP BY and HAVING clauses
3. **Joins**: Multiple table joins with intuitive syntax
4. **Models**: Reusable query components (CTEs)
5. **Window Functions**: Advanced analytical queries
6. **Advanced Features**: PIVOT, UNNEST, and more
7. **Target-Specific SQL**: Optimized for different databases
8. **Real-World Examples**: Complete analytical workflows
9. **Integration**: Using with pandas and DuckDB
10. **Error Handling**: Best practices for production use

### Next Steps

- Explore the [Wvlet documentation](https://wvlet.org/docs/)
- Try Wvlet with your own data and queries
- Integrate Wvlet into your data pipeline
- Contribute to the [Wvlet project](https://github.com/wvlet/wvlet)